In [1]:
import sys
sys.path.append('../')

In [2]:
import os

import numpy as np
from sklearn.tree import ExtraTreeClassifier
from sklearn.datasets import make_classification

from aad.datasets import DATASET_LIST, DataContainer
from aad.utils import get_data_path

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
NAME = 'Iris'

In [5]:
dc = DataContainer(DATASET_LIST[NAME], get_data_path())
dc(shuffle=True, normalize=True)

In [6]:
x_train = dc.x_train
y_train = dc.y_train
x_test = dc.x_test
y_test = dc.y_test

In [7]:
classifier = ExtraTreeClassifier(
    criterion='gini',
    splitter='random',
)
classifier.fit(x_train, y_train)

ExtraTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, random_state=None,
                    splitter='random')

In [8]:
print(classifier.feature_importances_)

[ 0.05617823  0.00148903  0.36513569  0.57719706]


In [9]:
pred = classifier.predict(x_test)

In [10]:
def evaluate(predictions, labels):
    return np.sum(np.equal(predictions, labels)) / len(labels)

In [11]:
accuracy = evaluate(pred, y_test)
print(f'Accuracy on test set: {accuracy}')

Accuracy on test set: 0.9


## Apply decision tree attack

In [12]:
from art.attacks import DecisionTreeAttack
from art.classifiers import SklearnClassifier

In [13]:
art_classifier = SklearnClassifier(classifier)
attack = DecisionTreeAttack(art_classifier)
adv = attack.generate(x_test)
pred_adv = classifier.predict(adv)

In [14]:
accuracy = evaluate(pred_adv, y_test)
print(f'Accuracy on DecisionTreeAttack: {accuracy}')

Accuracy on DecisionTreeAttack: 0.03333333333333333


## Test Transferability

In [15]:
attack_list = ['FGSM', 'BIM', 'DeepFool', 'Carlini']
x = np.load(os.path.join('..', 'save', 'IrisNN_Iris_BIM_x.npy'), allow_pickle=False)
y = np.load(os.path.join('..', 'save', 'IrisNN_Iris_BIM_y.npy'), allow_pickle=False)
pred = classifier.predict(x)
accuracy = evaluate(pred, y)
print(f'Accuracy on clean set: {accuracy}')

adv_files = []
for adv_name in attack_list:
    adv_files.append(f'IrisNN_Iris_{adv_name}_adv.npy')

Accuracy on clean set: 1.0


In [16]:
for i in range(len(attack_list)):
    adv = np.load(os.path.join('..', 'save', adv_files[i]), allow_pickle=False)
    pred = classifier.predict(adv)
    accuracy = evaluate(pred, y)
    print(f'Accuracy on {attack_list[i]} set: {accuracy}')

Accuracy on FGSM set: 0.38333333333333336
Accuracy on BIM set: 0.03333333333333333
Accuracy on DeepFool set: 0.43333333333333335
Accuracy on Carlini set: 0.35


## Applicability Domain

In [17]:
from aad.basemodels import ModelContainerTree
from aad.defences import ApplicabilityDomainContainer

In [20]:
mc = ModelContainerTree(classifier, dc)
hidden_model = None
ad = ApplicabilityDomainContainer(
    mc,
    mc.hidden_model,
    k2=6,
    reliability=1.0,
    sample_ratio=1.0,
    kappa=6,
    confidence=0.8
)
ad.fit()

True

In [22]:
blocked_idx = ad.detect(x, y)
print(f'Blocked {len(blocked_idx)}/{len(x)} samples on Clean set')

Blocked 8/60 samples on Clean set


In [ ]:
for i in range(len(attack_list)):
    adv = np.load(os.path.join('..', 'save', adv_files[i]), allow_pickle=False)
    blocked_idx = ad.detect(adv)
    print(f'Blocked {len(blocked_idx)}/{len(x)} samples on {attack_list[i]}')